<a href="https://colab.research.google.com/github/Ganeshs170220/AI-MODELS/blob/main/AI_GENERATE_QUESTION_AND_ANSWERS_FROM_URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AI GENERATE QUESTION AND ANSWERS FROM URL

#installing required modules

In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00


In [ ]:
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=7781e21950ba3cb8f85090e7950063763b933cfc23a94a6f3dcbf1e70694a953
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


#impoorting required modules

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
from transformers import pipeline
import requests
from bs4 import BeautifulSoup

#Get the URL

In [ ]:
URL = ("https://en.wikipedia.org/wiki/Narendra_Modi")
r = requests.get(URL)

#parse the data from URL

In [ ]:
soup = BeautifulSoup(r.text,'html.parser')
results = soup.find_all(['h1','p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [ ]:
ARTICLE

'Narendra Modi \n Premiership\n Budgets\n \n Campaigns\n Missions\n Establishments and foundations\n Events and observances\n Military and rescue operations\n Treaties and accords\n \n Establishments and foundations\n Events and observances\n Military and rescue operations\n Treaties and accords\n Controversies\n \n Narendra Damodardas Modi (Gujarati:\xa0[ˈnəɾendɾə dɑmodəɾˈdɑs ˈmodiː] (listen); born 17 September 1950)[b] is an Indian politician who has served as the 14th Prime Minister of India since May 2014. Modi was the Chief Minister of Gujarat from 2001 to 2014 and is the Member of Parliament (MP) for Varanasi. He is a member of the Bharatiya Janata Party (BJP) and of the Rashtriya Swayamsevak Sangh (RSS), a right-wing Hindu nationalist paramilitary volunteer organisation. He is the longest-serving non-Congress prime minister and the fourth longest-serving prime minister of all time.\n Modi was born and raised in Vadnagar in northeastern Gujarat, where he completed his secondary e

#Creating the chunk

In [ ]:
max_chunk = 500

#Data cleaning

In [ ]:
ARTICLE = ARTICLE.replace('\n','')
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')
ARTICLE = ARTICLE.replace('\\', '')

In [ ]:
ARTICLE

'Narendra Modi  Premiership Budgets  Campaigns Missions Establishments and foundations Events and observances Military and rescue operations Treaties and accords  Establishments and foundations Events and observances Military and rescue operations Treaties and accords Controversies  Narendra Damodardas Modi (Gujarati:\xa0[ˈnəɾendɾə dɑmodəɾˈdɑs ˈmodiː] (listen); born 17 September 1950)[b] is an Indian politician who has served as the 14th Prime Minister of India since May 2014.<eos><eos> Modi was the Chief Minister of Gujarat from 2001 to 2014 and is the Member of Parliament (MP) for Varanasi.<eos><eos> He is a member of the Bharatiya Janata Party (BJP) and of the Rashtriya Swayamsevak Sangh (RSS), a right-wing Hindu nationalist paramilitary volunteer organisation.<eos><eos> He is the longest-serving non-Congress prime minister and the fourth longest-serving prime minister of all time.<eos><eos> Modi was born and raised in Vadnagar in northeastern Gujarat, where he completed his seconda

#splitting the Data into chunks

In [ ]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


#length of the chunks

In [ ]:
len(chunks)

24

#Retrive the pre defined model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")
model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")

#AI Generating the question and answers from the chunks

In [ ]:
length = len(chunks)
count = 1
for i in range(0,length):
  inputs = tokenizer(chunks[i], return_tensors="pt")
  outputs = model.generate(**inputs, max_length=100)
  question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
  question_answer = question_answer.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
  question, answer = question_answer.split(tokenizer.sep_token)
  print(f'question-{count}: {question}')
  print("answer:", answer)
  print("--------------------------")
  count +=1

question-1:  What was the name of the Hindu nationalist paramilitary organization Modi is a member of?
answer:  Rashtriya Swayamsevak Sangh
--------------------------
question-2:  What was the name of the family that Modi was born into?
answer:  Gujarati Hindu
--------------------------
question-3:  What was the name of the Hindu ashram Modi visited in Almora?
answer:  Advaita Ashrama
--------------------------
question-4:  What was the name of the book Modi wrote in Gujarati?
answer:  Sangharsh Ma Gujarat
--------------------------
question-5:  What was the name of the BJP leader who defected to the Congress?
answer:  Shankersinh Vaghela
--------------------------
question-6:  What did the Supreme Court of India reopen in March 2008?
answer:  several cases related to the riots
--------------------------
question-7:  What did the Supreme Court uphold?
answer:  no evidence against him
--------------------------
question-8:  What did Modi's policies reduce in Gujarat?
answer:  corruption